Ансамбль двух моделей : предсказывающей настоящесть текста + предсказывающей какой из двух настоящий

In [15]:
import pandas as pd
import numpy as np

In [16]:
test_path = '/kaggle/input/fake-or-real-the-impostor-hunt/data/test'
train_path = '/kaggle/input/fake-or-real-the-impostor-hunt/data/train'
train_marks = '/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv'

In [17]:
# Код извлекающий тексты и создающий словарь {текст: метка(1 - real, 0 - fake)}
from pathlib import Path

data_dir = Path(train_path)
all_data = {}

for txt_file in data_dir.rglob("*.txt"):
    text_path = str(txt_file)
    # Вычленение из пути файла номера папки в которой хранится файл и номера текста (1 или 2)
    # Пример text_path : /kaggle/input/fake-or-real-the-impostor-hunt/data/train/article_0026/file_2.txt
    folder_num = int(text_path[text_path.find('article_') + 8 : text_path.find('article_') + 12])
    file_num = int(text_path[text_path.find('file_') + 5 : text_path.find('file_') + 6])

    content = txt_file.read_text(encoding='utf-8')

    all_data[(folder_num, file_num)] = content

marks = pd.read_csv(train_marks, index_col='id')
textes = {}

# 1 - Real, 0 - Fake
for (folder_num, text_num), text in all_data.items():
    if marks.iloc[folder_num]['real_text_id'] == text_num:
        textes[text] = 1
    else:
        textes[text] = 0

all_data = {}

for txt_file in data_dir.rglob("*.txt"):
    text_path = str(txt_file)
    # Вычленение из пути файла номера папки в которой хранится файл и номера текста (1 или 2)
    # Пример text_path : /kaggle/input/fake-or-real-the-impostor-hunt/data/train/article_0026/file_2.txt
    folder_num = int(text_path[text_path.find('article_') + 8 : text_path.find('article_') + 12])
    file_num = int(text_path[text_path.find('file_') + 5 : text_path.find('file_') + 6])

    content = txt_file.read_text(encoding='utf-8')

    if folder_num in all_data:
        all_data[folder_num][file_num] = content
    else:
        all_data[folder_num] = {file_num : content}


In [18]:
# Код прогоняющий все тексты через BERT для получения cls эмбеддинга
# и создающий матрицы X для каждой модели в ансамбле

from transformers import BertTokenizer, BertModel
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU не доступен, используется CPU")

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name).to(device)

features_arr_1 = []
features_arr_2 = []
marks_vec_1 = []
marks_vec_2 = []

for folder_num, textes_dict in all_data.items():
    text1 = textes_dict[1]
    text2 = textes_dict[2]
    
    input1 = tokenizer(text1, return_tensors = 'pt', padding=True, truncation=True, max_length=512).to(device)
    input2 = tokenizer(text2, return_tensors = 'pt', padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output1 = bert_model(**input1)
        output2 = bert_model(**input2)

    cls_emb1 = output1.last_hidden_state[0,0,:].cpu().numpy()
    cls_emb2 = output2.last_hidden_state[0,0,:].cpu().numpy()

    # Для формирования X первой модели
    features_arr_1.append(cls_emb1)
    features_arr_1.append(cls_emb2)
    marks_vec_1.append(textes[text1])
    marks_vec_1.append(textes[text2])

    # Для формирования X второй модели
    dif_feature = cls_emb1 - cls_emb2
    
    features_arr_2.append(dif_feature)
    marks_vec_2.append(marks.iloc[folder_num])

X1 = np.vstack(features_arr_1)
y1 = np.array(marks_vec_1)

X2 = np.vstack(features_arr_2)
y2 = np.array(marks_vec_2)

Используется GPU: Tesla T4


In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

# Предсказатель настоящести текстов
classifier_1 = RandomForestClassifier(n_estimators=1000)

classifier_1.fit(X1,y1)

# Предсказатель какой из текстов настоящий
classifier_2 = RandomForestClassifier(n_estimators=1000)
classifier_2.fit(X2,y2)

/tmp/ipykernel_78/2602048937.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier_2.fit(X2,y2)


RandomForestClassifier(n_estimators=1000)

In [28]:
# Обработка для теста

data_dir = Path(test_path)
all_data = {}

for txt_file in data_dir.rglob("*.txt"):
    text_path = str(txt_file)
    # Вычленение из пути файла номера папки в которой хранится файл и номера текста (1 или 2)
    # Пример text_path : /kaggle/input/fake-or-real-the-impostor-hunt/data/train/article_0026/file_2.txt
    folder_num = int(text_path[text_path.find('article_') + 8 : text_path.find('article_') + 12])
    file_num = int(text_path[text_path.find('file_') + 5 : text_path.find('file_') + 6])

    content = txt_file.read_text(encoding='utf-8')

    if folder_num in all_data:
        all_data[folder_num][file_num] = content
    else:
        all_data[folder_num] = {file_num : content}

marks = pd.read_csv(train_marks, index_col='id')

preds = []

for folder_num, textes_dict in all_data.items():
    text1 = textes_dict[1]
    text2 = textes_dict[2]
    
    input1 = tokenizer(text1, return_tensors = 'pt', padding=True, truncation=True, max_length=512).to(device)
    input2 = tokenizer(text2, return_tensors = 'pt', padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output1 = bert_model(**input1)
        output2 = bert_model(**input2)

    cls_emb1 = output1.last_hidden_state[0,0,:].cpu().numpy()
    cls_emb2 = output2.last_hidden_state[0,0,:].cpu().numpy()

    # Смотрим на настоящесть текстов
    text1_true_prob = classifier_1.predict_proba(cls_emb1.reshape(1,-1))[0][1]
    text2_true_prob = classifier_1.predict_proba(cls_emb2.reshape(1,-1))[0][1]

    # Предсказываем какой из текстов настоящий
    dif_feature = cls_emb1 - cls_emb2
    pred = classifier_2.predict_proba(dif_feature.reshape(1,-1))[0]

    # Суммируем предсказания моделей
    pred += np.array([text1_true_prob, text2_true_prob])

    if pred[0] > pred[1]:
        preds.append(1)
    else:
        preds.append(2)
    

In [29]:
submission = pd.DataFrame({'id' : all_data.keys(), 'real_text_id' : preds})
submission.to_csv('/kaggle/working/output.csv', index = False)